In [1]:
import pickle
from random import shuffle
import numpy as np
from statistics import mean

In [2]:
file_object = open('E:/Time Series/data_collect4.pkl','rb')
dicti = pickle.load(file_object)
file_object.close()
data2=  dicti['data']

In [3]:
data = data2

In [4]:
data = {key:value[:11000] for key, value in data.items() if len(value)>=11000}

In [5]:
for value in data.values():
    print(len(value))

11000
11000
11000
11000
11000


In [7]:
data = {key: [[np.uint16(i) for key2 in ['07','08','09','10','11','12'] for i in list(example[key2])] for example in value] for key, value in data.items()}

In [8]:
mins = {}; maxs = {}; means = {}
for i in range(30):
    lin = [l[i] for value in data.values() for l in value]
    mins[i] = min(lin)
    maxs[i] = max(lin)
    means[i] = mean(lin)
    
data = {key: [[np.float64((example[i]-means[i])/(maxs[i] - mins[i])) for i in range(30)] for example in value] for key, value in data.items()}

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in ushort_scalars
  


In [157]:
for key, value in data.items():
    lit = []
    for example in value:
        lis = []
        for key2 in ['07','08','09','10','11','12']:
            value2 = list(example[key2])
            for i in value2:
                lis.append(np.uint16(i))
        lit.append(lis)
    print(len(lit))
    data[key] = lit

11000
11000
11000
11000


In [ ]:
for i in range(30):
    lin = []
    for key, value in data.items():
        if key!= 'info':
            for l in value:
                lin.append(l[i])
    mins = min(lin)
    maxs = max(lin)
    means = mean(lin)
    for key, value in data.items():
        for example in value:
            example[i] = np.float64((example[i] - means)/(maxs-mins))

In [15]:
data = {key:[[example[i] for i in z] for example in value] for key, value in data.items()}

In [16]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:9000]
    cv_examples = cv_examples + value[9000:10000]
    test_examples = test_examples + value[10000:]
    training_labels = training_labels + [i for v in range(9000)]
    cv_labels = cv_labels + [i for v in range(1000)]
    test_labels = test_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1

0 : SOYABEENS
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : WATER BODY


In [17]:
from sklearn.utils import shuffle as shf
training_examples, training_labels  = shf(training_examples, training_labels)

In [13]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(training_examples + cv_examples, training_labels + cv_labels)
# display the relative importance of each attribute
print(model.feature_importances_)
s = model.feature_importances_
L = np.array(s)
t = sorted(range(len(L)), key=lambda i:L[i])
z = []
for j in t[::-1]:
    print( j, '-----', s[j])
    if s[j]>0.03: z.append(j)

[0.06127524 0.01398746 0.02196455 0.11467601 0.01987551 0.03240372
 0.02006415 0.07180373 0.02995107 0.0352105  0.01632578 0.0178532
 0.03576882 0.03444795 0.02759525 0.02549255 0.01623841 0.04256522
 0.02151836 0.07092703 0.02075564 0.01534277 0.03744998 0.03727137
 0.03314784 0.02801742 0.02649667 0.04449899 0.01173838 0.0153364 ]
3 ----- 0.11467601102590255
7 ----- 0.07180373016884643
19 ----- 0.0709270328113387
0 ----- 0.06127524379411612
27 ----- 0.04449899387502481
17 ----- 0.04256522367409158
22 ----- 0.037449983143291884
23 ----- 0.03727136532711462
12 ----- 0.035768820787592234
9 ----- 0.03521050075938585
13 ----- 0.03444795138216541
24 ----- 0.033147844044021
5 ----- 0.03240372051941482
8 ----- 0.02995107222948054
25 ----- 0.02801742069436499
14 ----- 0.027595253359223665
26 ----- 0.02649666785978133
15 ----- 0.025492545292751245
2 ----- 0.021964546403847762
18 ----- 0.021518356637655424
20 ----- 0.020755639418926105
6 ----- 0.020064151217103116
4 ----- 0.019875508082726995
1

In [ ]:
from sklearn import svm
clf = svm.SVC(C = 100000.0, tol = 0.00001, verbose = True)
clf.fit(training_examples, training_labels)

[LibSVM]

In [205]:
import pickle
file_object = open('clf.pkl', 'wb')
pickle.dump(clf, file_object)
file_object.close()

In [ ]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

In [ ]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

In [233]:
from sklearn.neural_network import MLPClassifier
cnf = MLPClassifier(alpha=1e-3,solver = 'sgd',
	hidden_layer_sizes=(30,15,10,4), 
	learning_rate_init = 0.001, 
	max_iter = 1000, 
	activation = 'tanh',
	batch_size = 100,
	tol = 0.000001,
    verbose = True
	)

In [234]:
cnf.fit(training_examples, training_labels)

Iteration 1, loss = 1.26807988
Iteration 2, loss = 1.19372981
Iteration 3, loss = 1.17899196
Iteration 4, loss = 1.17156842
Iteration 5, loss = 1.16493020
Iteration 6, loss = 1.15802281
Iteration 7, loss = 1.15362085
Iteration 8, loss = 1.15142365
Iteration 9, loss = 1.14988654
Iteration 10, loss = 1.14879949
Iteration 11, loss = 1.14786305
Iteration 12, loss = 1.14689943
Iteration 13, loss = 1.14631363
Iteration 14, loss = 1.14583534
Iteration 15, loss = 1.14492408
Iteration 16, loss = 1.14472470
Iteration 17, loss = 1.14384072
Iteration 18, loss = 1.14288318
Iteration 19, loss = 1.14243831
Iteration 20, loss = 1.14084054
Iteration 21, loss = 1.13933805
Iteration 22, loss = 1.13718415
Iteration 23, loss = 1.13477061
Iteration 24, loss = 1.13107668
Iteration 25, loss = 1.12712950
Iteration 26, loss = 1.12117981
Iteration 27, loss = 1.11405898
Iteration 28, loss = 1.10647516
Iteration 29, loss = 1.09826025
Iteration 30, loss = 1.09033091
Iteration 31, loss = 1.08165387
Iteration 32, los

MLPClassifier(activation='tanh', alpha=0.001, batch_size=100, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 15, 10, 4), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=1e-06, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [235]:
sum(cnf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.58975

In [236]:
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)

0.6173055555555556